<a href="https://colab.research.google.com/github/aditya248/Netflix-Exploratory-Data-Analysis/blob/main/BA775_TeamProject_B2T8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#BA775 - Team Project - Team 8
##*Project Proposal*

###**Summary Research for Netflix on it’s library**

Our mission is to help Netflix summarize the details of their movie and TV show library to help them see trends on how their library has changed. We will also summarize Netflix materials based on popular genres and actors to compare with highest earning  and popular entertainment types so Netflix can maximize their user amount. 

**Source:**
https://www.kaggle.com/shivamb/netflix-sh


In [5]:
from google.colab import auth
auth.authenticate_user()

##**Part 1 -** Overview of Netflix Library

In [ ]:
## Average duration of movie and TV shows -Hanyu

In [40]:
%%bigquery --project=ba775-team8-b2
SELECT type,AVG(new_duration) AS average_duration FROM
(SELECT type,CAST(REPLACE(REPLACE(REPLACE(duration,'min',''),'Season',''),'s','') AS INT64) AS new_duration
FROM `ba775-team8-b2.Netflix.netflix_titles`)
GROUP BY type

,type,average_duration
0,Movie,99.307978
1,TV Show,1.775934


In [ ]:
## Overview of Rating-Hanyu

In [59]:
%%bigquery --project=ba775-team8-b2
#For TV show
SELECT rating, COUNT(rating) AS count_rating FROM `ba775-team8-b2.Netflix.netflix_titles`
WHERE type='TV Show' AND rating IS NOT NULL
GROUP BY rating
ORDER BY 2 DESC

,rating,count_rating
0,TV-MA,1018
1,TV-14,659
2,TV-PG,301
3,TV-Y7,176
4,TV-Y,163
5,TV-G,83
6,NR,5
7,R,2
8,TV-Y7-FV,1


In [58]:
%%bigquery --project=ba775-team8-b2
SELECT rating, COUNT(rating) AS count_rating FROM `ba775-team8-b2.Netflix.netflix_titles`
WHERE type='Movie' AND rating IS NOT NULL
GROUP BY rating
ORDER BY 2 DESC 


,rating,count_rating
0,TV-MA,1845
1,TV-14,1272
2,R,663
3,TV-PG,505
4,PG-13,386
5,PG,247
6,TV-Y,117
7,TV-G,111
8,TV-Y7,95
9,NR,79


In [ ]:
## Movies/TV shows Released by Year -Hanyu

In [57]:
%%bigquery --project=ba775-team8-b2
SELECT release_year, COUNT(release_year) AS movie_added FROM `ba775-team8-b2.Netflix.netflix_titles`
GROUP BY release_year
ORDER BY release_year

,release_year,movie_added
0,1925,1
1,1942,2
2,1943,3
3,1944,3
4,1945,3
...,...,...
68,2017,1012
69,2018,1121
70,2019,996
71,2020,868


In [ ]:
## Popular Directors in Netflix -Hanyu

In [68]:
%%bigquery --project=ba775-team8-b2
SELECT director, COUNT(director) AS time_appeared
FROM `ba775-team8-b2.Netflix.netflix_titles`
WHERE director IS NOT NULL
GROUP BY 1
ORDER BY 2 DESC

,director,time_appeared
0,"Raúl Campos, Jan Suter",18
1,Marcus Raboy,16
2,Jay Karas,14
3,Cathy Garcia-Molina,13
4,Martin Scorsese,12
...,...,...
4044,"Max Lang, Jani Lachauer",1
4045,"Joe Murray, Cosmo Segurson",1
4046,Joel Crawford,1
4047,Mario Cambi,1


##**Part 2 -** Netflix Movies

In [ ]:
## INNER JOIN two dataset and find out the top 10 rating movies

In [ ]:
%%bigquery --project=ba775-team8-b2
SELECT n.title, n.director, n.country, n.date_added, r.Rating
FROM `ba775-team8-b2.Netflix.netflix_titles` as n
INNER JOIN `ba775-team8-b2.Netflix.IMBd_movies_ratings` as r
   ON lower(n.title) = lower(r.title)
ORDER BY r.Rating DESC
LIMIT 10

,title,director,country,date_added,Rating
0,Inception,Christopher Nolan,"United States, United Kingdom","January 1, 2020",8.8
1,Dangal,Nitesh Tiwari,India,"June 21, 2017",8.8
2,The Intouchables,"Olivier Nakache, Éric Toledano",France,"January 11, 2021",8.6
3,Taare Zameen Par,Aamir Khan,India,"January 10, 2021",8.5
4,The Departed,Martin Scorsese,"United States, Hong Kong","January 1, 2021",8.5
5,The Lives of Others,Florian Henckel von Donnersmarck,Germany,"March 15, 2019",8.5
6,3 Idiots,Rajkumar Hirani,India,"August 1, 2019",8.4
7,Django Unchained,Quentin Tarantino,United States,"April 25, 2020",8.4
8,Inglourious Basterds,Quentin Tarantino,"Germany, United States","July 22, 2019",8.3
9,Paint It Black,Amber Tamblyn,United States,"February 1, 2018",8.3


INNER JOIN two dataset and find out the highest grossing movie

In [ ]:
%%bigquery --project=ba775-team8-b2
SELECT n.title, n.director, n.country, n.date_added, r.Rating, r.Revenue__Millions_
FROM `ba775-team8-b2.Netflix.netflix_titles` as n
INNER JOIN `ba775-team8-b2.Netflix.IMBd_movies_ratings` as r
   ON lower(n.title) = lower(r.title)
ORDER BY r.Revenue__Millions_ DESC

,title,director,country,date_added,Rating,Revenue__Millions_
0,The Jungle Book,None,"India, Germany, France","May 11, 2019",7.5,364.00
1,Spider-Man 3,Sam Raimi,United States,"November 1, 2019",6.2,336.53
2,Indiana Jones and the Kingdom of the Crystal S...,Steven Spielberg,United States,"January 1, 2019",6.2,317.01
3,Inception,Christopher Nolan,"United States, United Kingdom","January 1, 2020",8.8,292.57
4,Star Trek,None,United States,"October 1, 2017",8.0,257.70
...,...,...,...,...,...,...
243,Paint It Black,Amber Tamblyn,United States,"February 1, 2018",8.3,NaN
244,"I, Daniel Blake",Ken Loach,"United Kingdom, France, Belgium","April 19, 2019",7.9,NaN
245,Tramps,Adam Leon,United States,"April 21, 2017",6.5,NaN
246,The Siege of Jadotville,Richie Smyth,"Ireland, South Africa","October 7, 2016",7.3,NaN


In [103]:
## Distribution of duration for Movies -Hanyu
%%bigquery --project=ba775-team8-b2
SELECT duration_range, COUNT(*) AS num_movies FROM 
(
SELECT CASE 
WHEN CAST(REPLACE(duration,'min','') AS INT64) <40 THEN '0-40'
WHEN CAST(REPLACE(duration,'min','') AS INT64) BETWEEN 40 AND 80 THEN '40-80'
WHEN CAST(REPLACE(duration,'min','') AS INT64) BETWEEN 80 AND 120 THEN '80-120'
WHEN CAST(REPLACE(duration,'min','') AS INT64) BETWEEN 120 AND 160 THEN '120-160'
WHEN CAST(REPLACE(duration,'min','') AS INT64) BETWEEN 160 AND 200 THEN '160-200'
ELSE '200+' END AS duration_range
FROM `ba775-team8-b2.Netflix.netflix_titles`
WHERE type='Movie'
)
GROUP BY duration_range
ORDER BY duration_range ###order???
 


,duration_range,num_movies
0,0-40,146
1,120-160,874
2,160-200,122
3,200+,15
4,40-80,848
5,80-120,3372


##**Part 3 -** Netflix TV Shows

In [ ]:
%%bigquery --ba775-team8-b2


##**Part 4 -** Trend over time

In [ ]:
%%bigquery --ba775-team8-b2


##**Part 5 -** Maximizing User Amount / Research Analysis Conclusion

In [ ]:
%%bigquery --ba775-team8-b2
